In [1]:
import requests 
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
import time
import re

In [2]:
query = '서유열'
url = 'https://search.daum.net/search?w=news&q='+query 

urls_list = [] #기사 url 담을 list

In [3]:
for i in range(10): 
    print (i)
    url=url+"&p="+str(i)

    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    for urls in source.find_all('a', {'class' : "f_nb"}):
        urls_list.append(urls["href"])

print(urls_list)
# urls_list[0]

0
1
2
3
4
5
6
7
8
9
['http://v.media.daum.net/v/20191223103054419', '?w=news&req=tab&q=%EC%84%9C%EC%9C%A0%EC%97%B4&viewio=i&repno=0&n=10&p=1&related_page=1&cluster_docid=26XBvgLSOWf_CCJal6', 'http://v.media.daum.net/v/20191220060311094', 'http://v.media.daum.net/v/20191220164805947', 'http://v.media.daum.net/v/20191122143218582', 'http://v.media.daum.net/v/20191122143234590', 'http://v.media.daum.net/v/20191122143233589', 'http://v.media.daum.net/v/20191122163709964', '?w=news&req=tab&q=%EC%84%9C%EC%9C%A0%EC%97%B4&viewio=i&repno=0&n=10&p=1&related_page=1&cluster_docid=26cE5Ah4HU6FZGXetM', 'http://v.media.daum.net/v/20191122173603808', 'http://v.media.daum.net/v/20191122154027953', 'http://v.media.daum.net/v/20191101100331489', 'http://v.media.daum.net/v/20191010113137033', 'http://v.media.daum.net/v/20191010131801794', 'http://v.media.daum.net/v/20191010154625605', '?w=news&req=tab&q=%EC%84%9C%EC%9C%A0%EC%97%B4&viewio=i&repno=0&n=10&p=1&related_page=1&cluster_docid=26doS71xahy2P7Btlz',

In [4]:
print(len(urls_list))

160


In [8]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time


driver = webdriver.Chrome('./chromedriver') 
comments=[] #댓글들 담을 list
for url in urls_list:
    try:
        print(url+' 크롤링 중')
        driver.get(url)
        time.sleep(3) #댓글 불러오기 기다림
        
        finish_flag = 1 # 댓글 더보기 버튼 클릭 횟수 제한하기
        while finish_flag == 1:
            try:
                driver.find_element_by_class_name('link_fold').click()
                time.sleep(3)# ip차단 방지
                
                driver.find_element_by_xpath('//*[@id="comment459089131"]/div/div/span[2]').click()
                time.sleep(2)
                    
                driver.find_element_by_xpath('//*[@id="comment443137110"]/div/div[2]/div[3]/a').click()
                time.sleep(1)
            
                
            except:
                finish_flag = 0 # false일 경우 멈추고 아래 코드로 이동.
        
                    
            html = driver.page_source

            bs = BeautifulSoup(html, 'html.parser')
            contents = bs.find_all("p", {"class" : "desc_txt font_size_17"}) #댓글 담긴 span tag 찾기
            for content in contents :
                print(content.text)
                comments.append(content.text)

    except:
        print("존재하지 않는 url입니다.")

        

# 크롤링 중 같은 페이지가 반복되는 오류 있음. 크롤링이 멈추지 않음. 
# 답댓글도 크롤링이 안됨.

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
len(comments) #댓글 수

In [ ]:
comments_str=" ".join(commens) #댓글 string으로

comments_str

In [ ]:
# comments_str 저장
save=open("daum_comments.txt", 'w' , encoding='utf-8' )
save.write(comments_str)
save.close()

In [ ]:
with open('daum_comments.txt', 'r', encoding= 'utf-8') as f: 
    doc = ''  # 리뷰 데이터를 담기 위한 String 변수 생성
    lines = f.readlines()  # 영화 리뷰 파일의 모든 라인을 읽어와 리스트로 저장
    for line in lines:  # for문을 통해 lines에 있는 모든 텍스트를 doc2에 이어 붙임
        doc += line

In [ ]:
doc

In [ ]:
from collections import Counter
from konlpy.tag import Twitter

# articles = df['Article'].tolist()
# articles = ''.join(articles)

twitter = Twitter()
raw_pos_tagged = twitter.pos(doc, norm=True, stem=True)

raw_pos_tagged

In [ ]:
test_str='완전 위해 절대 그게 아무'
test=twitter.pos(test_str, norm=True, stem=True)
print(test)

In [ ]:


del_list = ['하다', '있다', '되다', '이다', '돼다', '않다', '그렇다', '아니다', '이렇다', '그렇다','어떻다','ㅋㅋ','ㅋㅋㅋ','ㅠㅠ','ㅉㅉ','ㅎㅎ','없다','위해','완전','절대','그게','아무','건가','나다','모두','이미'] 
word_cleaned = []
for word in raw_pos_tagged:
    if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign","Adverb","Suffix"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
        if (len(word[0]) != 1) & (word[0] not in del_list): # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외
            if(word[0].isalpha()): #숫자 뺌
                word_cleaned.append(word[0])

word_counted = Counter(word_cleaned)
word_dic = dict(word_counted)
        
sorted_word_dic = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)

sorted_word_dic

In [ ]:
# 빈도 순으로 상위 100개에 해당하는 단어까지만 추려내어 번역을 진행합니다. (늘릴 수 있습니다.)

translation_target = sorted_word_dic[:100] 
translation_target[-1] # 상위 100번째에 해당하는 단어의 빈도수를 확인합니다. (가장 마지막 item을 호출하려면?)

In [ ]:
translation_target = {}
for item in word_dic: #item에 키값 들어감
    if word_dic[item] >= 31: # 위에서 확인한 빈도수를 기반으로 번역 대상 단어를 선정합니다.
        translation_target[item] = word_dic[item]

In [ ]:
print(translation_target)


In [ ]:
#워드 클라우드
# WordCloud 패키지를 import 합니다.
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image # 만약 "No module named 'PIL'" 에러가 발생하면 [ pip install Pillow==5.4.1 ] 로 라이브러리를 설치해줍니다.

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# WordCloud 객체를 생성합니다.
# 아래 보이는 코드의 font_path 에 해당하는 부분이 워드클라우드의 폰트에 적용됩니다. (대체로 고딕 계열 폰트가 이쁘게 나옵니다)
# 만약 다른 폰트를 적용하려면, C > Windows > Fonts 폴더에 원하시는 폰트 파일(.ttf)을 넣은 후 아래 코드에서 font 이름을 수정해줍니다.

# word_cloud = WordCloud().generate(text) : dict 가 아닌 줄글 텍스트 원문으로부터 워드클라우드를 만들 수 있습니다.
# word_cloud = WordCloud(font_path="fonts/malgun.ttf", # font_path="C:/Windows/Fonts/NanumSquareB.ttf"
#                        width=2000, height=1000, # 이 부분을 수정하시면 실제 워드클라우드의 크기가 바뀝니다 (해상도가 바뀝니다)
#                        # prefer_horizontal= 1.0, # 이 부분의 주석을 해제하시면 단어들이 가로로만 그려지게 됩니다. (0~1)
#                        background_color='white')

In [ ]:
# word_cloud.generate_from_frequencies(word_dic) # Generate wordcloud from word "frequency" data.

In [ ]:
# # 생성한 WordCloud 객체로부터 워드클라우드를 보여주는 코드입니다.

# plt.figure(figsize=(15,15)) # 이 부분을 수정하시면 화면에서 보여지는 워드클라우드의 크기가 바뀝니다 
# plt.imshow(word_cloud) # image show
# plt.axis("off")
# plt.tight_layout(pad=0)
# plt.show()

In [ ]:
# word_cloud.to_file("word_cloud.png")

In [ ]:
from wordcloud import ImageColorGenerator # Image 로부터 Color 를 생성(Generate)해내는 객체입니다.

coloring = np.array(Image.open("./yellow-house.png"))
image_colors = ImageColorGenerator(coloring)

word_cloud = WordCloud(font_path="fonts/malgun.ttf", # font_path="C:/Windows/Fonts/NanumSquareB.ttf"
                       width=2000, height=1000,  
                       mask=coloring, 
                       background_color='white').generate_from_frequencies(word_dic)

plt.figure(figsize=(15,15))
plt.imshow(word_cloud.recolor(color_func=image_colors), interpolation='bilinear') # 다시(re) 색칠하기
# plt.imshow(word_cloud.?(colormap='Blues'), interpolation='bilinear') # Matplotlib colormap 활용 (http://j.mp/32UXOQ6)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

In [ ]:
word_cloud.to_file("daum_word_cloud_coloring.png")

In [ ]:
coloring = np.array(Image.open("./yellow-house.png"))
image_colors = ImageColorGenerator(coloring)

word_cloud = WordCloud(font_path="fonts/malgun.ttf", # font_path="C:/Windows/Fonts/NanumSquareB.ttf"
                       width=2000, height=1000,  
                       mask=coloring, 
                       background_color='white').generate(doc)

plt.figure(figsize=(15,15)) # 이 부분을 수정하시면 화면에서 보여지는 워드클라우드의 크기가 바뀝니다 
plt.imshow(word_cloud) # image show
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()